In [ ]:
import os

import numpy
from matplotlib import pyplot as plt

from fit_diagnostics import abs_residuals, norm_residuals
from plot_utils import cplot, plot_red_vis
from red_likelihood import degVis, doRelCal, doOptCal, doDegVisVis, group_data, gVis, \
makeEArray, norm_rel_sols, red_ant_sep, relabelAnts, doRelCalRP
from red_utils import find_flag_file, find_nearest, find_zen_file, get_bad_ants, \
match_lst, split_rel_results

In [ ]:
numpy.set_printoptions(threshold=500)

In [ ]:
plt.rcParams['figure.figsize'] = (12, 8)
%matplotlib inline

In [ ]:
JD = 2458098.43869
pol = 'ee'
freq_channel = 300
time_integration = 0

In [ ]:
zen_fn = find_zen_file(JD)
bad_ants = get_bad_ants(zen_fn)
flags_fn = find_flag_file(JD, 'first') # import flags from firstcal

In [ ]:
hdraw, RedG, cMData = group_data(zen_fn, pol, freq_channel, None, bad_ants, flags_fn)
cData = cMData.filled() # filled with nans for flags
flags = cMData.mask

# mitigating for multiple freqs - only chooses first one
if cData.shape[0] > 1:
    cData = cData[0, ...]
    flags = flags[0, ...]
    print('Frequency channel {} selected for notebook analysis\n'.format(freq_channel[0]))
cData = numpy.squeeze(cData)
flags = numpy.squeeze(flags)

if all(numpy.isnan(cData[time_integration, :])):
    print('All visibilities for channel {} and time integration {} are flagged '\
          '- choose different values'.format(freq_channel, time_integration))

ants = numpy.unique(RedG[:, 1:])
no_unq_bls = numpy.unique(RedG[:, 0]).size

In [ ]:
plot_red_vis(cData, RedG, vis_type='amp')

# Redundant calibration

Fundamentally, the problem of calibration boils down to the measurement equation:

$$ V_{ij}^{\text{obs}} (\nu) = g_i (\nu) g_j^* (\nu) V_{ij}^{\text{true}}(\nu) + n_{ij} (\nu) $$

where the observed visibility $V_{ij}$ between antennas $i$ and $j$ at a given time and frequency is related to the true, underlying visibility by a pair of complex and frequency-dependent gain factors, $g_i$ and $g_j$, if we assume per-antenna gains, along with uncorrelated Gaussian random noise $n_{ij}$. The ultimate aim of calibration is to solve for these gains and visibilities. 

For a redundant array, we have a system of equations for all antenna pairs $i$ and $j$, given by

$$ V_{ij}^{\text{obs}} (\nu) = g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) $$

where $U_{\alpha}(\nu) = V(\mathbf{r}_i-\mathbf{r}_j)$, the visibility for the baseline vector $ \mathbf{b}_{ij} = \mathbf{r}_i-\mathbf{r}_j$, which corresponds to a redundant baseline set that we index by $\alpha$. Two pairs of identical elements with precisely the same baseline separation between them are sensitive to the same mode on the sky, and are thus measuring the same $V_{ij}^{\text{true}}$.

In highly redundant arrays, there are many more observations than there are unique baselines. For the full HERA array, there are 331 elements in the hexagonal core, corresponding to $N_{\mathrm{bl}} = 331(331-1)/2 = 54,615$ baselines. The hexagonal core only has 630 unique baseline separations, this means we have a non-linear system of $54,615$ equations to determine 630 unique visibilities and the 331 complex gains; the system is vastly overdetermined.

## Relative calibration

### Gaussian distribution

By imposing that the true sky visibilities from redundant baselines are equal, relative redundant calibration solves for the true sky visibilities for each redundant baseline set $V_{i-j}^{\text{sol}}$, alongside the gains of the individual antennas.

Assuming Gaussian uncorrelated noise with variance $\sigma_{ij}^2$, a maximum-likelihood estimate for the gains and sky visibilities can be constructed:

$$ \mathcal{L} (\{g_i(\nu)\}, \{U_{\alpha}(\nu)\} | \{V_{ij}^{\text{obs}}(\nu)\} \propto \prod_{\nu} \prod_{\alpha} \prod_{\{i,j\}_{\alpha}} \exp{\left( -\frac{1}{2} \left( \frac{ \left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^{*} (\nu) U_{\alpha}(\nu) \right|^2}{\sigma_{ij}^2(\nu)} \right) \right)} $$

where $\{i,j\}_{\alpha}$ are sets of antennas that belong to each redundant baseline type $\alpha$. Maximizing this function is equivalent to minimizing:

$$ \chi_{\mathrm{rel}}^{2} (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|^2 }{\sigma_{ij}^2(\nu)} $$

by varying $g_k(\nu)$ and $U_{\alpha}(\nu)$ for each frequency $\nu$.

This non-linear least-squares optimization can be done independently between frequencies and time.

### Cauchy distribution

We no longer assume that the noise in the measurement equation is Gaussian. Empirically, it is found that visibility observations can have a lot of outliers, and the visibilities follow a distribution with fatter tails than a Gaussian. We wish to be insensitive to outliers, so we need to employ robust statistics to adequately deal with such measurements.

Instead of fitting the visibilities to a Gaussian, we fit them to a Cauchy distribution, when solving the redundant calibration parameter estimation.

The Cauchy distribution is given by

$$ f(x; x_0, \gamma) = \frac{1}{\pi \gamma \left[ 1 + \left( \frac{x - x_0}{\gamma}  \right)^2  \right] }$$

where $x_0$ is the location parameter (the median) and $\gamma$ is the scale parameter, which specifies the half-width at half-maximum (HWHM).

Working in a maximum likelihood framework, the likelihood when solving for the measurement equation for redundant baseline sets is given by

$$ \mathcal{L} = \prod_{\nu} \prod_{\alpha} \prod_{\{i,j\}_{\alpha}} \frac{1}{\pi \gamma_{\alpha} (\nu)} \left[ 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|}{\gamma_{\alpha} (\nu)}  \right)^2  \right]^{-1} $$

The negative log-likelihood is therefore given by

$$ -\ln(\mathcal{L}) (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha} (\nu)) + \ln \left( 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - g_i (\nu) g_j^* (\nu) U_{\alpha}(\nu) \right|}{\gamma_{\alpha}(\nu)}  \right)^2 \right) $$

where we have dropped the sum over frequency, as we can solve independently for each frequency.

In [ ]:
obs_vis = cData[time_integration, :]
res_rel = doRelCal(RedG, obs_vis, distribution='cauchy')
res_rel_vis, res_rel_gains = split_rel_results(res_rel['x'], no_unq_bls)

In [ ]:
# Profiling methods

# import cProfile
# %load_ext line_profiler

# %lprun -f doRelCal doRelCal(RedG, obs_vis, distribution='cauchy')
# %prun doRelCal(RedG, obs_vis, distribution='cauchy')
# cProfile.run("doRelCal(RedG, obs_vis, distribution='cauchy')", sort='cumtime')

In [ ]:
# renormalize after solving
numpy.abs(res_rel_gains) / numpy.mean(numpy.abs(res_rel_gains))

In [ ]:
# Try with alternative calibration method whereby the number of parameters in the minimization
# is reduced such that the average amplitude and phase are constrained
res_relRP, _ = doRelCalRP(RedG, obs_vis, distribution='cauchy')
RP_vis, RP_gains = split_rel_results(res_relRP['x'], no_unq_bls, coords='polar')
RP_mean = numpy.mean(numpy.abs(RP_gains))
print('Average amp: {}, product of amps: {}, average phase: {}'.format(RP_mean, \
      numpy.prod(numpy.abs(RP_gains)), numpy.mean(numpy.angle(RP_gains))))

In [ ]:
# Residuals for relative redundant step
pred_rel_vis = gVis(res_rel_vis, relabelAnts(RedG), res_rel_gains)
rel_residuals = obs_vis - pred_rel_vis
cplot(rel_residuals, xlabel='Baseline', ylabel='Residual')

In [ ]:
# Relative residuals normalized by amplitude
norm_rel_residuals = norm_residuals(obs_vis, pred_rel_vis)
cplot(norm_rel_residuals, xlabel='Baseline', ylabel='Normalized residual')
print('Median absolute normalized residual - Real: {}, Imag: {}'.format(*abs_residuals(norm_rel_residuals)))

## Optimal calibration (solving for degeneracies)

Relative calibration yields degenerate solutions, that can be parameterized as four terms per frequency:
 - Overall amplitude $A(\nu)$
 - Overall phase $\Delta(\nu)$
 - Phase gradient components $\Delta_x(\nu)$ and $\Delta_y(\nu)$
 
The below transformations of these degenerate parameters leave $-\ln(\mathcal{L})$ (for both Gaussian and Cauchy distributions) unchanged:
 - $g_i \rightarrow A g_i$ accompanied by $U_{\alpha} \rightarrow A^{-2} U_{\alpha}$
 - $g_k = |g_k|e^{i\phi_k} \rightarrow |g_k|e^{i(\phi_k + \Delta)}$ corresponds to $g_k g_l^{*} = |g_k| |g_l| e^{i(\phi_k - \phi_l)} \rightarrow |g_k| |g_l| e^{i(\phi_k + \Delta - \phi_l - \Delta)} = g_k g_l^{*}$
 - $g_k = |g_k|e^{i\phi_k} \rightarrow |g_k|e^{i(\phi_k + \Delta_x x_k + \Delta_y y_k)}$ accompanied by $U_{\alpha} = |U_{\alpha}| e^{i\phi_{\alpha}} \rightarrow |U_{\alpha}| e^{i(\phi_{\alpha} - \Delta_x x_{\alpha} - \Delta_y y_{\alpha})}$
 
Where in the last line, the array is assumed to be co-planar, and $(x_k, y_k)$ are the coordinates of the position of antennas $k$, and $(x_{\alpha}, y_{\alpha})$ are the separations of the antennas that form baselines in redundant set $\alpha$.

These degeneracies must be constrained - this is the "absolute" part of redundant calibration. The degenerate parameters can either calculated from a sky model, or can be solved for using optimal absolute calibration, which calculates the degenerate parameters directly from the $\chi^2$ or $-\ln(\mathcal{L})$ by applying a few conditions.

 - Gaussian distribution:
 
$$ \chi_{\text{opt}}^2 (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left|  V_{ij}^{\text{obs}} (\nu) - h_i (\nu) h_j^{*} (\nu) W_{\alpha} (\nu) \right|^2 }{\sigma_{ij}^2(\nu)} $$

 - Cauchy distribution:

$$  -\ln(\mathcal{L}) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha} (\nu)) + \ln \left( 1 + \left( \frac{\left| V_{ij}^{\text{obs}} (\nu) - h_i (\nu) h_j^{*} (\nu) W_{\alpha} (\nu) \right|}{\gamma_{\alpha}(\nu)}  \right)^2 \right) $$

where

$$ W_{\alpha} (\nu) = A^2(\nu) e^{i \left[ \Delta_{x} (\nu) x_{\alpha} + \Delta_{y} (\nu) y_{\alpha} \right]} U_{\alpha} $$

with the following constraints applied to the $h_i$ parameters:

$$\frac{1}{N} \sum_i^N |h_i| = 1$$

$$\frac{1}{N} \sum_i^N \mathrm{Arg} (h_i) = 0$$

$$\sum_i^N x_i \mathrm{Arg} (h_i) = 0$$

$$\sum_i^N y_i \mathrm{Arg} (h_i) = 0$$

which translate to:
 - A mean gain amplitude of 1
 - A mean gain phase of 0
 - Phase gradients (also known as tilts) in both the x & y directions of 0
 
The overall phase is also degenerate, and is set by requiring that the phase of the gain of a reference antenna is null:

$$ \mathrm{Arg} (h_\text{ref}) = 0 $$

In [ ]:
res_opt = doOptCal(RedG, obs_vis, hdraw.antpos, res_rel_vis, distribution='cauchy', \
                   ref_ant=85)

In [ ]:
new_gain_params, new_deg_params = numpy.split(res_opt['x'], [ants.size*2,])
new_amps = new_gain_params[:ants.size*2:2]
new_phases = new_gain_params[1:ants.size*2:2]
new_gains = makeEArray(new_gain_params)

In [ ]:
print('Degenerate parameters: {}'.format(str(new_deg_params)[1: -1]))
print('Amplitude mean: {}'.format(numpy.mean(new_amps)))
print('Phase mean: {}'.format(numpy.mean(new_phases)))

In [ ]:
# Optimal residuals for optimal redundant step
ant_sep = red_ant_sep(RedG, hdraw.antpos)
opt_w_alpha = degVis(ant_sep, res_rel_vis, *new_deg_params[[0, 2, 3]])
pred_opt_vis = gVis(opt_w_alpha, relabelAnts(RedG), new_gains)
opt_residuals =  obs_vis - pred_opt_vis
cplot(opt_residuals, xlabel='Baseline', ylabel='Residual')

In [ ]:
# Normalized amplitude residuals
plt.figure(figsize=(12, 8))
plt.plot(norm_residuals(numpy.abs(obs_vis), numpy.abs(pred_opt_vis)))
plt.xlabel('Baseline')
plt.ylabel('Normalized residual')
plt.show()

In [ ]:
# Phase residuals
diff_phases = numpy.angle(obs_vis) - numpy.angle(pred_opt_vis)
# wrap between {-pi, pi}
diff_phases_wrapped = (diff_phases + numpy.pi) % (2 * numpy.pi) - numpy.pi
plt.figure(figsize=(12,8))
plt.plot(diff_phases)
plt.plot(diff_phases_wrapped, label='wrapped')
plt.xlabel('Baseline')
plt.ylabel('Normalized residual')
plt.legend()
plt.show()

In [ ]:
# Residuals normalized by amplitude
norm_opt_residuals = norm_residuals(obs_vis, pred_opt_vis)
cplot(norm_opt_residuals, xlabel='Baseline', ylabel='Residual')
print('Median absolute normalized residual - Real: {}, Imag: {}'.\
      format(*abs_residuals(norm_opt_residuals)))

# Comparing relative calibrations

To monitor the consistency of the "true" visibility solutions for each baseline set, we calculate the relative calibration visibility solutions from another JD and compare them to the relative solutions of the initial dataset. These should be consistent, up to the degenerate parameters $A$, $\Delta_x$ and $\Delta_y$ - solving for these parameters with an MLE framework enables us to compare these solutions.

We need to minimize:

- Assuming Gaussian distributions:

$$ \chi^2_{\text{deg}} (\nu) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \frac{ \left| U_{\alpha}' (\nu) - W_{\alpha} (\nu) \right|^2 }{\sigma_{\alpha}^2(\nu)} $$

- Assuming Cauchy distributions:

$$-\ln(\mathcal{L}) = \sum_{\alpha} \sum_{\{i,j\}_{\alpha}} \ln(\pi \gamma_{\alpha}) + \ln \left( 1 + \left( \frac{\left| U_{\alpha}' (\nu) - W_{\alpha} (\nu) \right|}{\gamma_{\alpha}}  \right)^2 \right) $$

In [ ]:
JD2 = match_lst(JD, 2458099, tint=time_integration) # finding the JD_time of the zen_file
# that matches the LAST of the dataset used in 1.

zen_fn2 = find_zen_file(JD2)
bad_ants2 = get_bad_ants(zen_fn2)
flags_fn2 = find_flag_file(JD2, 'first') # import flags from firstcal

In [ ]:
hdraw2, RedG2, cMData2 = group_data(zen_fn2, pol, freq_channel, None, bad_ants2, flags_fn2)
cData2 = cMData2.filled()
flags2 = cMData2.mask

if cData2.shape[0] > 1:
    cData2 = cData2[0, ...]
    flags2 = flags2[0, ...]
cData2 = numpy.squeeze(cData2)
flags2 = numpy.squeeze(flags2)  

no_unq_bls2 = numpy.unique(RedG2[:, 0]).size

print('Do the visibilities for JDs {} and {} have:\n'\
      'the same bad antennas? {}\n'\
      'the same flags? {}\n'\
      'the same redundant grouping? {}'.format(JD, JD2, (bad_ants == bad_ants2).all(), \
      (flags == flags2).all(), (RedG==RedG2).all()))

In [ ]:
# Find time integration in dataset 2 that corresponds to closest LST to that of dataset 1
# This currently assumes that dataset 2 contains the correct time integration...
time_integration2 = find_nearest(hdraw2.lsts, hdraw.lsts[time_integration])[1]

In [ ]:
plot_red_vis(cData2, RedG2, vis_type='amp')

In [ ]:
# Relative calibration for the 2nd dataset
res_rel2 = doRelCal(RedG2, cData2[time_integration2, :], distribution='cauchy')
res_rel_vis2, res_rel_gains2 = split_rel_results(res_rel2['x'], no_unq_bls2)

In [ ]:
# Digging into results further as there are discrepencies with results stored in rel_df*
# that do not produce the same degenerate fitting results, when the deg_cal script is run

In [ ]:
res_rel['fun']

In [ ]:
res_rel2['fun']

In [ ]:
# Checking if normalizing the solutions changes anything...
res_rel_vis, res_rel_gains = split_rel_results(norm_rel_sols(res_rel['x'], no_unq_bls), no_unq_bls)
res_rel_vis2, res_rel_gains2 = split_rel_results(norm_rel_sols(res_rel2['x'], no_unq_bls), no_unq_bls)

In [ ]:
numpy.abs(res_rel_vis)

In [ ]:
numpy.abs(res_rel_vis2)

In [ ]:
numpy.angle(res_rel_vis).sum()

In [ ]:
numpy.angle(res_rel_vis2).sum()

In [ ]:
# Translating between relatively calibrated visibility sets
res_deg = doDegVisVis(RedG, hdraw.antpos, res_rel_vis, res_rel_vis2, distribution='cauchy')
deg_tr_params = res_deg['x']
print(deg_tr_params)

In [ ]:
res_deg['fun']

In [ ]:
# Residuals for degenerate comparison
deg_w_alpha = degVis(ant_sep, res_rel_vis, *deg_tr_params)
deg_residuals = res_rel_vis2 - deg_w_alpha
cplot(deg_residuals, xlabel='Redundant baseline type', ylabel='Residual')

In [ ]:
# Degenerate residuals normalized by amplitude
norm_deg_residuals = norm_residuals(res_rel_vis2, deg_w_alpha)
cplot(norm_deg_residuals, xlabel='Redundant baseline type', ylabel='Residual')
print('Median absolute normalized residual - Real: {}, Imag: {}'.format(*abs_residuals(norm_deg_residuals)))